In [2]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from ipywidgets import Image
from IPython import display
import numpy as np
from scipy.spatial import Delaunay
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
%matplotlib widget

In [7]:
w1data = pd.read_csv("nfl-big-data-bowl-2023/week1.csv")
players_background = pd.read_csv("BigDataBowl2023/data2023/players.csv", index_col='nflId')
madden_df = pd.read_csv("BigDataBowl2023/merged_df.csv")
median_df = pd.read_csv("BigDataBowl2023/all_pos_df.csv", index_col=0)

In [4]:
def collect_game_info(weekdf, playId, saved=False):
    
    #slice df to single play
    playdf = weekdf.loc[weekdf["playId"] == playId]
    #reorder playdf so that they are in the right frame order
    playdf = playdf.sort_values(by = "frameId")
    if saved:
        playdf.to_csv(f'Play{playId}Frames.csv')
    return playdf

In [23]:
def gaussian(x):
    return np.exp(-(((x**2)-(2*x)+1)/(1.2*x)))

In [25]:
print(gaussian(1))

1.0


In [11]:
def find_best_player(weekdf,playId,resolution,madden_df,median_df,players_background):
    play = collect_game_info(weekdf, playId)
    madden_df.loc[:, 'nflId'] = madden_df['nflId'].astype('int64')
    madden_df = madden_df.set_index('nflId')
    play_df = play.dropna()
    ball_df = play[play['team'] == 'football']
    play_df.loc[:, 'frameId'] = play_df['frameId'].astype('int')
    play_df.loc[:, 'nflId'] = play_df['nflId'].astype('int64')
    player_df = play_df.set_index(['nflId', 'frameId'])
    ball_df = ball_df.set_index(['frameId'])

        # Stuff for plotting
    num_frames = len(play_df['frameId'].unique())
        #self.num_players = len(self.play_df['jerseyNumber'].unique())
    teams = player_df['team'].unique()
    #xrange = (player_df['x'].min()-10, player_df['x'].max()+10)
    #yrange = (player_df['y'].min()-10, player_df['y'].max()+10)
    player_best_scores = {}
    player_speeds = {}
    player_catchs = {}
    
    ##megauhohthis is so bad
    #getting attacker and defener names
    idx = pd.IndexSlice
    player_frame_df = player_df.loc[idx[:, 1+1], :].droplevel(-1)
    for nflId, player in player_frame_df.iterrows():
            pos = players_background.loc[nflId]['officialPosition']
            if pos =='QB':
                qb_Id = nflId
                qb_team = player['team']
                qbx, qby = player[['x', 'y']]
    xrange = (player_df['x'].min(), player_df['x'].max()+10)
    yrange = (player_df['y'].min()-10, player_df['y'].max()+10)
    attackers = player_frame_df[player_frame_df['team']==qb_team]
    defenders = player_frame_df[player_frame_df['team']!=qb_team]
    for nflId, player in attackers.iterrows():#collecting speed scores + generating dicts of info
                player_best_scores[nflId] = 0
                player_position = players_background.loc[nflId]['officialPosition']
                if nflId in madden_df.index:
                                player_stats = madden_df.loc[nflId]
                else:
                                player_stats = median_df.loc[player_position]
                player_speeds[nflId] = player_stats['Speed'] + player_stats['Acceleration'] + player_stats['Agility']/3
                player_catchs[nflId] = player_stats['Catching']
    for nflId, player in defenders.iterrows():
                player_position = players_background.loc[nflId]['officialPosition']
                if nflId in madden_df.index:
                                player_stats = madden_df.loc[nflId]
                else:
                                player_stats = median_df.loc[player_position]
                player_speeds[nflId] = player_stats['Speed'] + player_stats['Acceleration'] + player_stats['Agility']/3


    for frame in range(10,num_frames,3):
        i = frame
        idx = pd.IndexSlice
        player_frame_df = player_df.loc[idx[:, i+1], :].droplevel(-1)
        ball_frame_df = ball_df.loc[i+1].to_frame().transpose() # the hatred towards pandas I have in this instant is unquantifiable with words
        ball_frame_df['nflId'] = 'football'
        ball_frame_df.set_index('nflId', inplace=True)

        # Find the nflId of the QB (theres probably a better way to do this but idk it)
        for nflId, player in player_frame_df.iterrows():
            pos = players_background.loc[nflId]['officialPosition']
            if pos =='QB':
                qb_Id = nflId
                qb_team = player['team']
                qbx, qby = player[['x', 'y']]


        # Sort the players into attackers and defenders
        attackers = player_frame_df[player_frame_df['team']==qb_team]
        defenders = player_frame_df[player_frame_df['team']!=qb_team]

        receiver_ratings = {'nflId':[], 'receiver rating':[]}

        qb_stats = madden_df.loc[qb_Id]

        throw_score = qb_stats['Throw On The Run'] + qb_stats['Throw Under Pressure'] + qb_stats['Throw Power'] # !!!wowowowow

        for x in np.arange(xrange[0],xrange[1]+resolution,resolution):#loop over xcoords
                for y in np.arange(yrange[0],yrange[1]+resolution,resolution):#loop over ycoords
                    min_dist = 1000000 # big
                    score = 0 # how good the space is too pass to
                    for nflId, player in defenders.iterrows():
                        dist_2_point = np.sqrt((x-player['x'])**2+ (y-player['y'])**2) / player_speeds[nflId] # weighted distance using speeds
                        if dist_2_point < min_dist:
                            min_dist = dist_2_point#results in smallest dist from defending player
                    for nflId, player in attackers.iterrows():
                        player_position = players_background.loc[nflId]['officialPosition']
                        dist_2_point = np.sqrt((x-player['x'])**2+ (y-player['y'])**2)/ player_speeds[nflId]
                        if dist_2_point < min_dist: #only if attacking player is closer to point than defending player
                            min_dist = dist_2_point
                            throw_dist = np.sqrt((x-qbx)**2 + (y-qby)**2)
                            
                            
                            if throw_dist >= 20:
                                throw_score += qb_stats['Throw Acc Deep']
                            elif throw_dist >=10:
                                throw_score += qb_stats['Throw Acc Mid']
                            else:
                                throw_score += qb_stats['Throw Acc Short']
                            # Average the throw score over the 4 metrics used
                            throw_score /= 4 # !!!
                            #score defined as: yard gain * 'chance of catching' (losely defined as throwscore X catchscore) * tanh(time ball takes to reach point/ time player takes to reach points)
                            #tanh thing a metric of will the player reach the catching zone before the ball if both going at max speed
                            score = (x-qbx)*  throw_score*player_catchs[nflId]*np.tanh((throw_dist*qb_stats['Throw Power'])/(min_dist*player_speeds[nflId]/3))#throw speed : player speed = roughly 3:1
                            if score > player_best_scores[nflId]:
                                player_best_scores[nflId] = score
            #for Id in player_best_scores:
           #     receiver_ratings['nflId'].append(Id)
           #     receiver_ratings['receiver rating'].append(player_best_scores[Id])
    for val in player_best_scores:
        print(player_best_scores[val])
        

find_best_player(w1data,90,1,madden_df,median_df,players_background)

    

26752.30666666667
3627.8999975973375
36099.93333330281
6816.509999999919
5118.244462886489
39829.89751576743
38372.67000000002
1791.0000000000082
11069.920000000002
1373.1000000000063
12873.641223292243
